El objetivo de este enfoque, sera generar un/os modelo/s para predecir los atributos del dia siguiente al ultimo disponible en el dataset. Aqui se aplicaran 2 enfoques:

- Un modelo que prediga todas las variablse en simultaneo (con el objetivo de captar la interrelacion entre las mismas).
- Un modelo que prediga solamente la variable target (incialmente se realizaran pruebas con la variable Close, y luego se procedera a usar la variable Tendencia).

Una vez realiza la prediccion de los atributos del dia siguiente, se procedera a realizar la prediccion de la Tendencia/Close, se realimientara el dataset, y se procedere a predecir otro dia, repitiendo esto N veces.

### Imports

In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Add
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score
from pmdarima.arima import auto_arima
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from neuralprophet import NeuralProphet
from tensorflow.keras.layers import Reshape
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import datetime
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from sklearn.model_selection import GridSearchCV
from sklearn.base import RegressorMixin
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import make_scorer
from keras.callbacks import Callback
from sklearn.base import clone
from sklearn.ensemble import VotingRegressor
from tensorflow.keras.regularizers import l2
from skopt import BayesSearchCV
import tensorflow.keras.backend as K

pd.set_option('display.max_columns', None)

2024-04-09 08:10:14.410479: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# LSTM Predictor

### Dataset

In [2]:
columns = [
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

In [3]:
dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')

dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
dates = dataset['Open_time'][:-5]

# dataset.drop(['Sentimiento'], axis=1, inplace=True)
# dataset.drop(['Sentimiento_coin'], axis=1, inplace=True)
# dataset.drop(['Sentimiento_referentes'], axis=1, inplace=True)
# dataset.drop(columns=['Open_time'], inplace=True)

dataset = dataset.round(2) # Limitar los valores float a 2 decimales en todo el dataframe

feature_dataset = dataset[columns]
# feature_dataset.drop(['Tendencia'], axis=1, inplace=True)

validation = feature_dataset[-5:]
feature_dataset = feature_dataset[:-5]

n_days_to_predict = 5

In [4]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,28.84,30.26,27.50,27.71,449178.0,42147.35,39776.84,1001487.0,2925.59,510130.73,1043885.0,335.5,956544.07,457187.0,31.85,30.83,38.30,31.85,25.39,44.11,0.0,0.0,0.0,139,135,1.0,81.0,525.0,364.0,270000.0
1,27.72,28.38,26.14,26.31,362304.0,41026.54,43372.26,1045389.0,2804.91,511325.46,928494.0,333.0,922077.23,417006.0,31.77,30.40,38.44,31.77,25.11,41.83,4.0,2.0,5273.0,93,122,2.0,87.0,472.0,331.0,204000.0
2,26.31,28.59,26.11,27.28,376232.0,41524.28,33511.53,884909.0,2850.45,411305.09,748804.0,367.7,1696420.04,653011.0,31.65,30.10,38.55,31.65,24.74,43.99,22.0,40.0,54144.0,112,145,0.0,64.0,594.0,495.0,216000.0
3,27.28,28.99,27.13,28.62,339737.0,43824.10,46381.23,1197815.0,3000.61,506896.76,992243.0,387.5,1163674.21,551245.0,31.62,29.96,38.56,31.62,24.67,46.92,15.0,23.0,39220.0,116,147,2.0,77.0,419.0,464.0,202000.0
4,28.61,32.33,28.50,31.94,735059.0,48141.61,66244.87,1771237.0,3309.91,648714.62,1446386.0,421.5,1440336.04,727854.0,31.64,30.15,38.58,31.64,24.69,53.42,24.0,35.0,63183.0,171,141,1.0,71.0,477.0,664.0,492000.0


(903, 30)


,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
903,10.08,10.46,9.60,9.90,245319.0,67609.99,55691.08,2464515.0,3520.46,570901.29,1906387.0,555.4,2284301.81,994512.0,10.06,9.95,11.86,10.06,8.26,52.48,34.0,43.0,84706.0,696,471,0.0,43.0,343.0,228.0,154000.0
904,9.90,9.99,8.60,8.77,341363.0,61937.40,101005.32,3593832.0,3158.64,1049629.69,2647385.0,507.7,2551361.51,1213572.0,10.08,9.84,11.81,10.08,8.35,42.93,120.0,126.0,135180.0,961,509,1.0,56.0,534.0,433.0,221000.0
905,8.77,9.57,8.49,9.48,267797.0,67840.51,90420.59,3549793.0,3516.53,1207322.82,2987953.0,556.8,1425296.58,809335.0,10.14,9.80,11.68,10.14,8.60,49.21,185.0,117.0,112997.0,866,555,1.0,40.0,473.0,386.0,171000.0
906,9.48,9.58,9.07,9.18,156774.0,65501.27,53357.48,2388390.0,3492.85,602755.21,1791989.0,553.8,953921.37,563996.0,10.17,9.74,11.63,10.17,8.71,46.85,64.0,81.0,66543.0,692,533,0.0,24.0,350.0,290.0,101000.0
907,9.18,9.37,8.69,8.94,147578.0,63796.64,51482.38,2492881.0,3336.35,558848.89,1747756.0,553.8,1181298.51,712381.0,10.14,9.67,11.67,10.14,8.62,45.00,57.0,66.0,68616.0,681,546,0.0,41.0,252.0,206.0,92000.0


(5, 30)

### Scalers

In [5]:
scalers = {}
for col in feature_dataset.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    feature_dataset[col] = scaler.fit_transform(np.array(feature_dataset[col]).reshape(-1, 1))
    scalers[col] = scaler


In [6]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,0.501993,0.517208,0.492593,0.479569,0.168108,0.460212,0.040979,0.046312,0.506304,0.126579,0.285966,0.303395,0.197596,0.238897,0.633771,0.615034,0.648008,0.633771,0.602961,0.345342,0.000000,0.000000,0.000000,0.123457,0.136662,0.058824,0.536424,0.302018,0.212508,0.140182
1,0.479673,0.480653,0.464609,0.451664,0.134757,0.440649,0.045762,0.049256,0.474653,0.126913,0.248880,0.297919,0.189790,0.215055,0.631960,0.605239,0.650664,0.631960,0.595284,0.307138,0.016878,0.007092,0.026035,0.079772,0.119580,0.117647,0.576159,0.268600,0.191449,0.104869
2,0.451574,0.484737,0.463992,0.470999,0.140104,0.449336,0.032645,0.038494,0.486597,0.098880,0.191129,0.373932,0.365156,0.355091,0.629244,0.598405,0.652751,0.629244,0.585138,0.343331,0.092827,0.141844,0.267329,0.097816,0.149803,0.000000,0.423841,0.345523,0.296107,0.111289
3,0.470905,0.492514,0.484979,0.497708,0.126093,0.489479,0.049765,0.059477,0.525979,0.125672,0.269369,0.417306,0.244504,0.294707,0.628565,0.595216,0.652941,0.628565,0.583219,0.392426,0.063291,0.081560,0.193644,0.101614,0.152431,0.117647,0.509934,0.235183,0.276324,0.103799
4,0.497409,0.557457,0.513169,0.563883,0.277858,0.564840,0.076188,0.097929,0.607100,0.165421,0.415326,0.491785,0.307160,0.399500,0.629018,0.599544,0.653321,0.629018,0.583767,0.501340,0.101266,0.124113,0.311958,0.153846,0.144547,0.058824,0.470199,0.271753,0.403957,0.258962


(903, 30)


,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
903,10.08,10.46,9.60,9.90,245319.0,67609.99,55691.08,2464515.0,3520.46,570901.29,1906387.0,555.4,2284301.81,994512.0,10.06,9.95,11.86,10.06,8.26,52.48,34.0,43.0,84706.0,696,471,0.0,43.0,343.0,228.0,154000.0
904,9.90,9.99,8.60,8.77,341363.0,61937.40,101005.32,3593832.0,3158.64,1049629.69,2647385.0,507.7,2551361.51,1213572.0,10.08,9.84,11.81,10.08,8.35,42.93,120.0,126.0,135180.0,961,509,1.0,56.0,534.0,433.0,221000.0
905,8.77,9.57,8.49,9.48,267797.0,67840.51,90420.59,3549793.0,3516.53,1207322.82,2987953.0,556.8,1425296.58,809335.0,10.14,9.80,11.68,10.14,8.60,49.21,185.0,117.0,112997.0,866,555,1.0,40.0,473.0,386.0,171000.0
906,9.48,9.58,9.07,9.18,156774.0,65501.27,53357.48,2388390.0,3492.85,602755.21,1791989.0,553.8,953921.37,563996.0,10.17,9.74,11.63,10.17,8.71,46.85,64.0,81.0,66543.0,692,533,0.0,24.0,350.0,290.0,101000.0
907,9.18,9.37,8.69,8.94,147578.0,63796.64,51482.38,2492881.0,3336.35,558848.89,1747756.0,553.8,1181298.51,712381.0,10.14,9.67,11.67,10.14,8.62,45.00,57.0,66.0,68616.0,681,546,0.0,41.0,252.0,206.0,92000.0


(5, 30)

### Preparo el dataset para train: cada conjunto de entrenamiento, sera una seried de N dias previos, para predecir 1 dia siguiente.

In [7]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        end_ix = i + n_steps
        seq_x = data.iloc[i:end_ix, :].values
        seq_y = data.iloc[end_ix, :].values
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


n_steps = 30  # Longitud de la secuencia de entrada
n_features = feature_dataset.shape[1]  # Número de características

# Crear las secuencias de entrada y salida
X, y = create_sequences(feature_dataset, n_steps)

In [8]:
print(X[0].shape) # Cada dato de entrenamiento, es un conjunto de 30 dias con sus 64 features
print(y[0].shape) # El target de cada dato, son los 64 features del dia siguiente
print(X.shape)
print(y.shape)
print(feature_dataset.shape)

(30, 30)
(30,)
(873, 30, 30)
(873, 30)
(903, 30)


In [9]:
print(len(X))
print(len(y))

873
873


### Obtencion de los mejores hiperparametros

In [10]:
def custom_scoring(estimator, X, y):
    y_pred = estimator.predict(X)
    mse = mean_squared_error(y, y_pred)
    return -mse

In [11]:
def custom_scoring_validation(y, y_pred):
    mse = mean_squared_error(y, y_pred)
    return -mse

In [12]:
def vmse(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis=-1)

In [13]:
# Ultimos cambios:
    # Se agrego early stopping
    # Se agregaron los optimizers a la optimizacion bayesiana
    # Se cambio la funcion de loss de mse a mae
    # Se cambio la arquitectura de la red:
        # dado que agrego batchNormalization, se omite dropout
        # Cambio la cantidad de units (Ahora todas las capas tienen la misma cantidad)
        # La profundida de la red es parte de la optimizacion

import warnings
warnings.filterwarnings('ignore')

def create_model(activation, units, dropout, learning_rate, l2_penalty, depth, optimizer='adam'):
    model = Sequential()
    model.add(Conv1D(units, kernel_size=3, activation=activation, input_shape=(n_steps, n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    for _ in range(depth - 1):
        model.add(Conv1D(units, kernel_size=3, activation=activation))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())

    # model.add(Conv1D(units, kernel_size=3, activation=activation))
    # model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout))
    
    # model.add(Conv1D(units, kernel_size=3, activation=activation))
    # model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(units, activation=activation, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(units=n_features))
    
    # model.add(Conv1D(units, kernel_size=kernel_size, activation=activation, input_shape=(n_steps, n_features)))
    # model.add(BatchNormalization())
    
    # for _ in range(depth - 1):
    #     model.add(Conv1D(units, kernel_size=kernel_size, activation=activation))
    #     model.add(BatchNormalization())
    #     model.add(MaxPooling1D(pool_size=2))
        
    # model.add(Flatten())
    # model.add(Dense(units, activation=activation, kernel_regularizer=l2(l2_penalty)))
    # model.add(BatchNormalization())
    
    # # Capa de salida
    # model.add(Dense(units=n_features))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss=vmse, metrics=['accuracy'])
    return model

regressor = KerasRegressor(build_fn=create_model, verbose=0, activation='relu', units=50, dropout=0.2, learning_rate=0.1, l2_penalty=0.001, depth=2, optimizer='adam')
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

cv = TimeSeriesSplit(n_splits=20).split(X)
param_space = {
    'depth': [2, 3],
    # 'kernel_size': [3, 6, 9],
    'activation': ['relu', 'tanh', 'swish', 'selu'],
    'units': [64, 128, 256, 512],
    'dropout': [0.1, 0.2, 0.3, 0.4],
    'learning_rate': [0.01, 0.001, 0.0001],
    'epochs': [10, 20, 30, 50, 100],
    'batch_size': [32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'l2_penalty': [0.001, 0.01, 0.1]
}

bayes_search = BayesSearchCV(regressor, param_space, scoring=custom_scoring, cv=cv, verbose=0)
bayes_result = bayes_search.fit(X, y, callbacks = [early_stopping])

In [14]:
# Show best results
print("Best score:", bayes_result.best_score_)
print("Best parameters:", bayes_result.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = bayes_result.best_estimator_
best_model.fit(X, y)

Best score: -0.014945430524693826
Best parameters: OrderedDict([('activation', 'tanh'), ('batch_size', 32), ('depth', 3), ('dropout', 0.4), ('epochs', 50), ('l2_penalty', 0.01), ('learning_rate', 0.001), ('optimizer', 'rmsprop'), ('units', 256)])


KerasRegressor(
	model=None
	build_fn=<function create_model at 0x7fec388cc430>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=50
	activation=tanh
	units=256
	dropout=0.4
	learning_rate=0.001
	l2_penalty=0.01
	depth=3
)

### Predicciones con el mejor conjunto de hiper parametros

In [15]:
n_days_to_predict = 5
future_dataset = feature_dataset

dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')
dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
dates = dataset['Open_time']

# Crear un DataFrame vacío para almacenar las predicciones desnormalizadas
predicted_values_desnormalized = pd.DataFrame(columns=future_dataset.columns)

# Lista para almacenar las fechas de las predicciones
predicted_dates = []

for _ in range(n_days_to_predict):
    # Predecir 1 día posterior al último día disponible en el dataset
    last_sequence = future_dataset.iloc[-n_steps:, :].values.reshape((1, n_steps, n_features))
    predictions = best_model.predict(last_sequence)

    # Agregar las predicciones sin desnormalizar a future_dataset
    predicted_values_normalized = pd.DataFrame(predictions, columns=future_dataset.columns)
    future_dataset = pd.concat([future_dataset, predicted_values_normalized], axis=0, ignore_index=True)

    # Desnormalizar las predicciones y agregarlas al DataFrame de predicciones desnormalizadas
    inverted_predictions = []
    for i in range(len(future_dataset.columns)):
        col = future_dataset.columns[i]
        scaler = scalers[col]
        prediction = predictions[:, i].reshape(-1, 1)
        inverted_prediction = scaler.inverse_transform(prediction)
        inverted_predictions.append(inverted_prediction)

    # Calcular la fecha del próximo día
    next_day_date = dates.iloc[-1] + pd.DateOffset(days=1)
    predicted_dates.append(next_day_date)

    # Actualizar la fecha del próximo día en el DataFrame principal
    dates = dates.append(pd.Series([next_day_date], name='Fecha'))

    # Crear un DataFrame con las predicciones desnormalizadas
    predicted_values_desnormalized = pd.concat([predicted_values_desnormalized,
                                                pd.DataFrame(np.concatenate(inverted_predictions, axis=1),
                                                             columns=future_dataset.columns)], 
                                                ignore_index=True)

# Agregar las fechas al DataFrame de predicciones desnormalizadas
predicted_values_desnormalized['Fecha'] = predicted_dates

print("Valores predichos para los próximos {} días:".format(n_days_to_predict))
display(future_dataset.tail(n_days_to_predict + 1))

print("Valores predichos desnormalizados para los próximos {} días:".format(n_days_to_predict))
display(predicted_values_desnormalized.tail(n_days_to_predict))


Valores predichos para los próximos 5 días:


,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
902,0.120167,0.128135,0.115844,0.128164,0.083843,0.918331,0.054236,0.143388,0.694908,0.128726,0.503699,0.820811,0.368893,0.443672,0.138751,0.139408,0.147628,0.138751,0.127228,0.515583,0.151899,0.170213,0.352479,0.649573,0.501971,0.000000,0.139073,0.156999,0.156988,0.075976
903,0.216350,0.220856,0.211053,0.214732,0.112964,0.744083,0.031945,0.094362,0.640497,0.107084,0.357486,0.517440,0.265600,0.319145,0.235087,0.233429,0.236812,0.231657,0.229314,0.699489,0.234968,0.178600,0.304918,0.229351,0.195754,-0.010150,0.258718,0.219295,0.204923,0.102679
904,0.214328,0.218792,0.209039,0.212682,0.111027,0.738971,0.027178,0.087148,0.635627,0.107122,0.354030,0.511703,0.263615,0.315115,0.233745,0.232036,0.235348,0.230351,0.227855,0.687336,0.232499,0.176383,0.300742,0.228404,0.194411,-0.010622,0.256129,0.216171,0.202121,0.101571
905,0.212247,0.216683,0.207221,0.210532,0.108251,0.731350,0.027154,0.085571,0.631571,0.106158,0.349996,0.511481,0.259843,0.310257,0.233107,0.231206,0.234637,0.229793,0.227220,0.668682,0.223106,0.168567,0.295021,0.226070,0.191471,-0.010513,0.253226,0.209816,0.196194,0.100164
906,0.215067,0.219538,0.209976,0.213259,0.109169,0.730516,0.028306,0.086549,0.631552,0.106105,0.349718,0.511914,0.259653,0.311586,0.236259,0.234281,0.237696,0.233032,0.230266,0.667151,0.222224,0.168242,0.295925,0.225689,0.191646,-0.009903,0.254774,0.208854,0.195157,0.100893
907,0.221527,0.226156,0.216614,0.219731,0.109972,0.729608,0.025555,0.081932,0.636190,0.103340,0.345416,0.518616,0.257550,0.311893,0.244986,0.242795,0.246227,0.241778,0.238888,0.652290,0.213998,0.161509,0.294880,0.223726,0.189790,-0.010177,0.256519,0.201002,0.188033,0.102510


Valores predichos desnormalizados para los próximos 5 días:


,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Fecha
0,14.506426,15.018613,13.817194,14.423111,305539.62500,58410.621094,32985.585938,1718050.000,3437.248779,440577.53125,1266416.875,433.211151,1256823.500,592430.9375,14.236122,14.077535,16.629971,14.084594,11.763090,65.245491,55.687397,50.365238,61757.210938,250.507095,179.969131,-0.172558,39.066448,393.802399,352.114471,199907.078125,2024-03-23
1,14.404958,14.912470,13.719303,14.320273,300493.12500,58117.792969,29402.314453,1610471.625,3418.681885,440711.46875,1255664.125,430.592621,1248058.375,585638.6875,14.176838,14.016373,16.552858,14.026902,11.709874,64.520233,55.102333,49.740128,60911.347656,249.509583,178.947006,-0.180570,38.675510,388.846832,347.723755,197836.781250,2024-03-24
2,14.300566,14.803983,13.630942,14.212396,293261.09375,57681.125000,29384.275391,1586953.125,3403.215576,437273.37500,1243112.750,430.490967,1231401.875,577451.3125,14.148661,13.979933,16.515362,14.002253,11.686726,63.406956,52.876198,47.535912,59752.738281,247.051743,176.709747,-0.178716,38.237087,378.767517,338.435791,195205.921875,2024-03-25
3,14.442062,14.950838,13.764856,14.349216,295653.71875,57633.351562,30249.919922,1601540.375,3403.144287,437083.21875,1242247.000,430.688934,1230562.000,579691.4375,14.287920,14.114918,16.676605,14.145339,11.797796,63.315590,52.667156,47.444298,59935.675781,246.650360,176.842712,-0.168358,38.470821,377.241974,336.810974,196568.171875,2024-03-26
4,14.766240,15.291197,14.087419,14.673927,297744.15625,57581.359375,28182.140625,1532676.375,3420.827637,427220.31250,1228861.000,433.748383,1221279.000,580207.7500,14.673493,14.488687,17.126171,14.531748,12.112259,62.428688,50.717510,45.545666,59724.125000,244.583847,175.430252,-0.173012,38.734337,364.789307,325.648376,199591.484375,2024-03-27


### Guardado de los mejores hiperparametros

In [16]:
import json

# Obtener los hiperparámetros y puntajes de los 5 mejores modelos
top_n_models = 5
best_params_list = []
best_scores_list = []

for i in range(min(top_n_models, len(bayes_search.cv_results_['params']))):
    best_params_list.append(bayes_search.cv_results_['params'][i])
    best_scores_list.append(bayes_search.cv_results_['mean_test_score'][i])

# Guardar los hiperparámetros de los 5 mejores modelos en un archivo JSON
with open('top_5_hyperparameters_conv.json', 'w') as f:
    json.dump({'best_params': best_params_list, 'best_scores': best_scores_list}, f)

# O imprimir los hiperparámetros
print("Top 5 mejores modelos:")
for i in range(len(best_params_list)):
    print("Modelo", i+1)
    print("Hiperparámetros:", best_params_list[i])
    print("Puntaje:", best_scores_list[i])


Top 5 mejores modelos:
Modelo 1
Hiperparámetros: OrderedDict([('activation', 'selu'), ('batch_size', 64), ('depth', 3), ('dropout', 0.1), ('epochs', 20), ('l2_penalty', 0.1), ('learning_rate', 0.001), ('optimizer', 'sgd'), ('units', 64)])
Puntaje: -0.038629554049147294
Modelo 2
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 128), ('depth', 3), ('dropout', 0.3), ('epochs', 100), ('l2_penalty', 0.01), ('learning_rate', 0.01), ('optimizer', 'sgd'), ('units', 512)])
Puntaje: -0.03132398189558662
Modelo 3
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 128), ('depth', 2), ('dropout', 0.2), ('epochs', 30), ('l2_penalty', 0.1), ('learning_rate', 0.0001), ('optimizer', 'rmsprop'), ('units', 128)])
Puntaje: -0.12186648494558452
Modelo 4
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 32), ('depth', 2), ('dropout', 0.4), ('epochs', 10), ('l2_penalty', 0.001), ('learning_rate', 0.0001), ('optimizer', 'rmsprop'), ('units', 64)])
Punta

### Armado de un ensamble con los mejores 5 hiperparametros usando la mejor semilla en cada caso

In [17]:
def generate_prime_seeds(n):
    seeds = []
    num = 70001  # Comenzamos desde el primer número primo mayor que 70000
    while len(seeds) < n:
        is_prime = True
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            seeds.append(num)
        num += 1
    return seeds


In [18]:
def predict_next_days(ensemble, feature_dataset, scalers, n_steps, n_features, n_days_to_predict):
    future_dataset = feature_dataset.copy()

    # Leer el conjunto de datos original para obtener las fechas
    dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')
    dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
    dates = dataset['Open_time'][:-n_days_to_predict]

    # Crear un DataFrame vacío para almacenar las predicciones desnormalizadas
    predicted_values_desnormalized = pd.DataFrame(columns=future_dataset.columns)

    # Lista para almacenar las fechas de las predicciones
    predicted_dates = []

    for _ in range(n_days_to_predict):
        # Predecir 1 día posterior al último día disponible en el dataset
        last_sequence = future_dataset.iloc[-n_steps:, :].values.reshape((1, n_steps, n_features))
        predictions = ensemble.predict(last_sequence)

        # Agregar las predicciones sin desnormalizar a future_dataset
        predicted_values_normalized = pd.DataFrame(predictions, columns=future_dataset.columns)
        future_dataset = pd.concat([future_dataset, predicted_values_normalized], axis=0, ignore_index=True)

        # Desnormalizar las predicciones y agregarlas al DataFrame de predicciones desnormalizadas
        inverted_predictions = []
        for i in range(len(future_dataset.columns)):
            col = future_dataset.columns[i]
            scaler = scalers[col]
            prediction = predictions[:, i].reshape(-1, 1)
            inverted_prediction = scaler.inverse_transform(prediction)
            inverted_predictions.append(inverted_prediction)

        # Calcular la fecha del próximo día
        next_day_date = dates.iloc[-1] + pd.DateOffset(days=1)
        predicted_dates.append(next_day_date)

        # Actualizar la fecha del próximo día en el DataFrame principal
        dates = dates.append(pd.Series([next_day_date], name='Fecha'))

        # Crear un DataFrame con las predicciones desnormalizadas
        predicted_values_desnormalized = pd.concat([predicted_values_desnormalized,
                                                    pd.DataFrame(np.concatenate(inverted_predictions, axis=1),
                                                                 columns=future_dataset.columns)], 
                                                    ignore_index=True)

    # Agregar las fechas al DataFrame de predicciones desnormalizadas
    predicted_values_desnormalized['Fecha'] = predicted_dates

    return future_dataset, predicted_values_desnormalized

In [19]:
## Clase personalizada para hacer el ensamble, dado que sklearn no provee ninguna clase que permita hacer ensmble
## de modelos re regresion multivariados
class MultivariableVotingRegressor:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)

    def predict(self, X):
        # Hacer predicciones con cada modelo
        predictions = [model.predict(X) for model in self.models]
    
        # Calcular el promedio de las predicciones
        average_predictions = np.mean(predictions, axis=0)
    
        return average_predictions


In [20]:

import json

# Leer los hiperparámetros desde el archivo JSON
with open('top_5_hyperparameters_conv.json', 'r') as f:
    top_hyperparameters = json.load(f)


models = []
best_seeds= {}
prime_seeds = generate_prime_seeds(300)

for mode_number, params in enumerate(top_hyperparameters['best_params']):
    best_validation_errors = {}
    
    for seend_number, seed in enumerate(prime_seeds):
        model = KerasRegressor(build_fn=create_model, random_state=seed, verbose=0, **params)
        
        model.fit(X, y)
        
        model_predictions, _ = predict_next_days(model, feature_dataset, scalers, n_steps, n_features, 5)

        error = custom_scoring_validation(validation, model_predictions[-5:])
        print(f"model number: {mode_number}, seed number: {seend_number} error: {error}")
        
        if seed not in best_validation_errors or error < best_validation_errors[seed]:
            best_validation_errors[seed] = error
    
    best_seed_for_params = min(best_validation_errors, key=best_validation_errors.get)
    best_seeds[str(params)] = best_seed_for_params
    
    model = KerasRegressor(build_fn=create_model, random_state=best_seed_for_params, verbose=0, **params)
    model.fit(X, y)
    models.append(model)


ensemble = MultivariableVotingRegressor(models)
ensemble.fit(X, y)

with open('best_seeds_conv.json', 'w') as f:
    json.dump(best_seeds, f)

model number: 1, seed number: 222 error: -622831645509.9629
model number: 1, seed number: 223 error: -622831641481.4478
model number: 1, seed number: 224 error: -622831633742.571
model number: 1, seed number: 225 error: -622831650830.6962
model number: 1, seed number: 226 error: -622831609538.0438
model number: 1, seed number: 227 error: -622831625760.3048
model number: 1, seed number: 228 error: -622831629757.7859
model number: 1, seed number: 229 error: -622831652072.9731
model number: 1, seed number: 230 error: -622831629212.7938
model number: 1, seed number: 231 error: -622831665008.1115
model number: 1, seed number: 232 error: -622831657502.8982
model number: 1, seed number: 233 error: -622831580407.6605
model number: 1, seed number: 234 error: -622831653860.5012
model number: 1, seed number: 235 error: -622831649453.6666
model number: 1, seed number: 236 error: -622831615368.9042
model number: 1, seed number: 237 error: -622831653945.9086
model number: 1, seed number: 238 error: 

In [21]:
future_dataset = feature_dataset

future_dataset, predicted_values_desnormalized = predict_next_days(ensemble, feature_dataset, scalers, n_steps, n_features, n_days_to_predict)

print("Valores predichos para los próximos {} días:".format(n_days_to_predict))
display(future_dataset.tail(n_days_to_predict + 1))

print("Valores predichos desnormalizados para los próximos {} días:".format(n_days_to_predict))
display(predicted_values_desnormalized.tail(n_days_to_predict))


Valores predichos para los próximos 5 días:


,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
902,0.120167,0.128135,0.115844,0.128164,0.083843,0.918331,0.054236,0.143388,0.694908,0.128726,0.503699,0.820811,0.368893,0.443672,0.138751,0.139408,0.147628,0.138751,0.127228,0.515583,0.151899,0.170213,0.352479,0.649573,0.501971,0.000000,0.139073,0.156999,0.156988,0.075976
903,0.058166,0.071053,0.093088,0.209103,0.185831,0.464913,0.177570,-0.265770,0.453063,0.272907,-0.046448,0.128281,0.044855,0.040945,0.214738,0.054068,0.019642,0.249467,0.253264,0.384640,0.117167,0.131928,0.099171,0.148628,0.198215,-0.080837,0.057983,0.256941,0.086346,-0.031942
904,0.053865,0.059676,0.107928,0.199356,0.184252,0.469203,0.210182,-0.245169,0.449417,0.268409,-0.053798,0.115805,0.066775,0.029250,0.218476,0.032941,0.009195,0.222476,0.266412,0.398104,0.117548,0.121060,0.119892,0.119854,0.189500,-0.074527,0.063870,0.241400,0.077387,0.013364
905,0.058332,0.096123,0.121772,0.178222,0.158552,0.475065,0.232054,-0.236756,0.430063,0.290036,-0.062651,0.131211,0.051065,0.004363,0.227632,0.011619,-0.009679,0.229419,0.275967,0.381162,0.142746,0.123641,0.122658,0.132233,0.202292,-0.073110,0.074081,0.242383,0.066761,0.047663
906,0.039550,0.119542,0.142280,0.177967,0.162377,0.460846,0.237408,-0.216928,0.441667,0.277357,-0.064674,0.119679,0.022562,-0.005907,0.201207,-0.026751,-0.026805,0.199530,0.294347,0.382179,0.140136,0.104527,0.153506,0.104307,0.190286,-0.105188,0.088579,0.263886,0.064236,0.034172
907,0.032229,0.133685,0.169269,0.173267,0.170423,0.472926,0.219638,-0.244406,0.424583,0.276831,-0.071239,0.109653,0.022438,-0.017584,0.234318,-0.011293,-0.032044,0.180447,0.307913,0.380740,0.134958,0.144291,0.126089,0.095902,0.197616,-0.091180,0.064725,0.260005,0.063938,0.021819


Valores predichos desnormalizados para los próximos 5 días:


,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Fecha
0,6.568762,7.314273,8.084068,14.140716,495344.40625,42416.695312,142456.609375,-3652509.00,2722.593018,1.032206e+06,9588.093750,255.560089,282103.78125,123575.445312,13.337129,6.203595,5.185119,14.871467,12.636547,46.455318,27.768692,37.203564,20085.880859,165.505447,181.841339,-1.374227,8.755414,453.509003,166.304886,-51699.308594,2024-03-18
1,6.352944,6.729131,8.805325,13.651693,491231.65625,42662.476562,166972.312500,-3345285.25,2708.688477,1.016160e+06,-13281.995117,249.865051,378897.12500,103865.914062,13.502258,5.276132,4.634593,13.678970,13.116061,47.258823,27.858992,34.138977,24282.658203,135.206604,175.209488,-1.266967,9.644383,428.860809,152.266159,32977.546875,2024-03-19
2,6.577085,8.603601,9.478132,12.591394,424287.28125,42998.308594,183414.578125,-3219832.75,2634.895752,1.093318e+06,-40826.648438,256.898010,309525.28125,61923.730469,13.906770,4.340058,3.639909,13.985720,13.464526,46.247765,33.830688,34.866688,24842.693359,148.241348,184.944519,-1.242866,11.186186,430.419220,135.614014,97081.687500,2024-03-20
3,5.634637,9.808044,10.474800,12.578601,434249.28125,42183.648438,187439.062500,-2924145.50,2679.139404,1.048083e+06,-47120.707031,251.633316,183667.21875,44615.621094,12.739328,2.655619,2.737377,12.665237,14.134830,46.308434,33.212223,29.476658,31090.611328,118.835098,175.807266,-1.788198,13.375440,464.523315,131.658295,71866.585938,2024-03-21
4,5.267235,10.535395,11.786478,12.342828,455209.37500,42875.769531,174081.140625,-3333910.75,2614.000977,1.046208e+06,-67547.328125,247.056595,183121.46875,24934.687500,14.202191,3.334253,2.461271,11.822153,14.629593,46.222549,31.985155,40.690067,25537.763672,109.984833,181.385696,-1.550068,9.773538,458.367767,131.190948,48780.425781,2024-03-22


### Rearmado del modelo a partir de las semillas

In [22]:
with open('best_seeds_conv.json', 'r') as f:
    best_seeds = json.load(f)

# 21 Crear y entrenar los modelos con los hiperparámetros y semillas guardados
models = []
for params_str, seed in best_seeds.items():
    params = json.loads(params_str.replace("'", "\""))
    model = KerasRegressor(build_fn=create_model, random_state=seed, **params)
    model.fit(X, y)
    models.append(model)

Epoch 1/20
14/14 [==============================] - 5s 17ms/step - loss: 8.4303 - accuracy: 0.0435
Epoch 2/20
14/14 [==============================] - 0s 16ms/step - loss: 6.3861 - accuracy: 0.0607
Epoch 3/20
14/14 [==============================] - 0s 12ms/step - loss: 4.9430 - accuracy: 0.0687
Epoch 4/20
14/14 [==============================] - 0s 19ms/step - loss: 3.7974 - accuracy: 0.0745
Epoch 5/20
14/14 [==============================] - 0s 13ms/step - loss: 2.9156 - accuracy: 0.1042
Epoch 6/20
14/14 [==============================] - 0s 11ms/step - loss: 2.2294 - accuracy: 0.1065
Epoch 7/20
14/14 [==============================] - 0s 14ms/step - loss: 1.6980 - accuracy: 0.1294
Epoch 8/20
14/14 [==============================] - 0s 15ms/step - loss: 1.2883 - accuracy: 0.1363
Epoch 9/20
14/14 [==============================] - 0s 12ms/step - loss: 0.9759 - accuracy: 0.1558
Epoch 10/20
14/14 [==============================] - 0s 10ms/step - loss: 0.7365 - accuracy: 0.1523
Epoch 11/

In [23]:
# 2. Predecir 5 días en el futuro con los modelos entrenados
ensemble = MultivariableVotingRegressor(models)
ensemble.fit(X, y)
future_dataset, predicted_values_desnormalized = predict_next_days(ensemble, feature_dataset, scalers, n_steps, n_features, 5)
    
display(predicted_values_desnormalized.head())

Epoch 1/20
14/14 [==============================] - 3s 12ms/step - loss: 8.4303 - accuracy: 0.0435
Epoch 2/20
14/14 [==============================] - 0s 16ms/step - loss: 6.3861 - accuracy: 0.0607
Epoch 3/20
14/14 [==============================] - 0s 11ms/step - loss: 4.9430 - accuracy: 0.0687
Epoch 4/20
14/14 [==============================] - 0s 11ms/step - loss: 3.7974 - accuracy: 0.0745
Epoch 5/20
14/14 [==============================] - 0s 10ms/step - loss: 2.9156 - accuracy: 0.1042
Epoch 6/20
14/14 [==============================] - 0s 10ms/step - loss: 2.2294 - accuracy: 0.1065
Epoch 7/20
14/14 [==============================] - 0s 11ms/step - loss: 1.6980 - accuracy: 0.1294
Epoch 8/20
14/14 [==============================] - 0s 13ms/step - loss: 1.2883 - accuracy: 0.1363
Epoch 9/20
14/14 [==============================] - 0s 11ms/step - loss: 0.9759 - accuracy: 0.1558
Epoch 10/20
14/14 [==============================] - 0s 10ms/step - loss: 0.7365 - accuracy: 0.1523
Epoch 11/

,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Fecha
0,6.568762,7.314273,8.084068,14.140716,495344.40625,42416.695312,142456.609375,-3652509.00,2722.593018,1.032206e+06,9588.093750,255.560089,282103.78125,123575.445312,13.337129,6.203595,5.185119,14.871467,12.636547,46.455318,27.768692,37.203564,20085.880859,165.505447,181.841339,-1.374227,8.755414,453.509003,166.304886,-51699.308594,2024-03-18
1,6.352944,6.729131,8.805325,13.651693,491231.65625,42662.476562,166972.312500,-3345285.25,2708.688477,1.016160e+06,-13281.995117,249.865051,378897.12500,103865.914062,13.502258,5.276132,4.634593,13.678970,13.116061,47.258823,27.858992,34.138977,24282.658203,135.206604,175.209488,-1.266967,9.644383,428.860809,152.266159,32977.546875,2024-03-19
2,6.577085,8.603601,9.478132,12.591394,424287.28125,42998.308594,183414.578125,-3219832.75,2634.895752,1.093318e+06,-40826.648438,256.898010,309525.28125,61923.730469,13.906770,4.340058,3.639909,13.985720,13.464526,46.247765,33.830688,34.866688,24842.693359,148.241348,184.944519,-1.242866,11.186186,430.419220,135.614014,97081.687500,2024-03-20
3,5.634637,9.808044,10.474800,12.578601,434249.28125,42183.648438,187439.062500,-2924145.50,2679.139404,1.048083e+06,-47120.707031,251.633316,183667.21875,44615.621094,12.739328,2.655619,2.737377,12.665237,14.134830,46.308434,33.212223,29.476658,31090.611328,118.835098,175.807266,-1.788198,13.375440,464.523315,131.658295,71866.585938,2024-03-21
4,5.267235,10.535395,11.786478,12.342828,455209.37500,42875.769531,174081.140625,-3333910.75,2614.000977,1.046208e+06,-67547.328125,247.056595,183121.46875,24934.687500,14.202191,3.334253,2.461271,11.822153,14.629593,46.222549,31.985155,40.690067,25537.763672,109.984833,181.385696,-1.550068,9.773538,458.367767,131.190948,48780.425781,2024-03-22


# Light GBM Classifier